# **Projeto Integrado Feature Engineering**





In [12]:
# Importação das bibliotecas necessárias para manipulação de dados, visualização e técnicas de machine learning.
import numpy as np
import pandas as pd

In [13]:
# Carregando o conjunto de dados que será utilizado para o projeto.

df_descritivo = pd.read_excel(r"descritivo_características.xlsx",engine='openpyxl')
dataset = pd.read_csv("./dataset.txt", sep="\t", encoding="latin1")
dataset.columns = df_descritivo['Característica'].tolist()

<ipython-input-13-277260586d04>:4: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv("./dataset.txt", sep="\t", encoding="latin1")


## **Data Understanding & Cleaning**

In [14]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 0 to 49998
Data columns (total 54 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ID_CLIENTE                         49999 non-null  int64  
 1   TIPO_FUNCIONARIO                   49999 non-null  object 
 2   DIA_PAGAMENTO                      49999 non-null  int64  
 3   TIPO_ENVIO_APLICACAO               49999 non-null  object 
 4   QUANT_CARTOES_ADICIONAIS           49999 non-null  int64  
 5   TIPO_ENDERECO_POSTAL               49999 non-null  int64  
 6   SEXO                               49999 non-null  object 
 7   ESTADO_CIVIL                       49999 non-null  int64  
 8   QUANT_DEPENDENTES                  49999 non-null  int64  
 9   NIVEL_EDUCACIONAL                  49999 non-null  int64  
 10  ESTADO_NASCIMENTO                  49999 non-null  object 
 11  CIDADE_NASCIMENTO                  49999 non-null  obj

In [15]:
dataset.head()

,ID_CLIENTE,TIPO_FUNCIONARIO,DIA_PAGAMENTO,TIPO_ENVIO_APLICACAO,QUANT_CARTOES_ADICIONAIS,TIPO_ENDERECO_POSTAL,SEXO,ESTADO_CIVIL,QUANT_DEPENDENTES,NIVEL_EDUCACIONAL,ESTADO_NASCIMENTO,CIDADE_NASCIMENTO,NACIONALIDADE,ESTADO_RESIDENCIAL,CIDADE_RESIDENCIAL,BAIRRO_RESIDENCIAL,FLAG_TELEFONE_RESIDENCIAL,CODIGO_AREA_TELEFONE_RESIDENCIAL,TIPO_RESIDENCIA,MESES_RESIDENCIA,FLAG_TELEFONE_MOVEL,FLAG_EMAIL,RENDA_PESSOAL_MENSAL,OUTRAS_RENDAS,FLAG_VISA,FLAG_MASTERCARD,FLAG_DINERS,FLAG_AMERICAN_EXPRESS,FLAG_OUTROS_CARTOES,QUANT_CONTAS_BANCARIAS,QUANT_CONTAS_BANCARIAS_ESPECIAIS,VALOR_PATRIMONIO_PESSOAL,QUANT_CARROS,EMPRESA,ESTADO_PROFISSIONAL,CIDADE_PROFISSIONAL,BAIRRO_PROFISSIONAL,FLAG_TELEFONE_PROFISSIONAL,CODIGO_AREA_TELEFONE_PROFISSIONAL,MESES_NO_TRABALHO,CODIGO_PROFISSAO,TIPO_OCUPACAO,CODIGO_PROFISSAO_CONJUGE,NIVEL_EDUCACIONAL_CONJUGE,FLAG_DOCUMENTO_RESIDENCIAL,FLAG_RG,FLAG_CPF,FLAG_COMPROVANTE_RENDA,PRODUTO,FLAG_REGISTRO_ACSP,IDADE,CEP_RESIDENCIAL_3,CEP_PROFISSIONAL_3,ROTULO_ALVO_MAU=1
0,2,C,15,Carga,0,1,F,2,0,0,RJ,rio de janeiro,1,RJ,RIO DE JANEIRO,CAMPO GRANDE,Y,20,1.0,1.0,N,1,750.0,0.0,0,0,0,0,0,0,0,0.0,0,Y,,NaN,NaN,N,,0,11.0,4.0,11.0,NaN,0,0,0,0,1,N,34,230,230,1
1,3,C,5,Web,0,1,F,2,0,0,RN,GARANHUNS,1,RN,Parnamirim,Boa Esperanca,Y,105,1.0,NaN,N,1,500.0,0.0,0,0,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,11.0,NaN,NaN,NaN,0,0,0,0,1,N,27,591,591,0
2,4,C,20,Web,0,1,F,2,0,0,PE,CABO,1,PE,CABO,PONTE DOS CARVALHOS,N,,NaN,NaN,N,1,500.0,0.0,0,0,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,NaN,NaN,NaN,NaN,0,0,0,0,1,N,61,545,545,0
3,5,C,10,Web,0,1,M,2,0,0,RJ,RIO DE JANEIRO,1,RJ,Rio de Janeiro,Santa Cruz,Y,20,1.0,12.0,N,1,1200.0,0.0,0,0,0,0,0,0,0,0.0,0,N,,NaN,NaN,N,,0,9.0,5.0,NaN,NaN,0,0,0,0,1,N,48,235,235,1
4,6,C,10,0,0,1,M,2,0,0,MG,CARMO DO RIO CLARO,1,MG,ALFENAS,Campos Eliseos,Y,33,1.0,4.0,N,1,500.0,0.0,0,0,0,0,0,1,1,0.0,1,Y,MG,ALFENAS,JARDIM SAO CARLOS,N,,0,9.0,2.0,0.0,0.0,0,0,0,0,2,N,40,371,371,1


In [16]:
dataset.describe()

,ID_CLIENTE,DIA_PAGAMENTO,QUANT_CARTOES_ADICIONAIS,TIPO_ENDERECO_POSTAL,ESTADO_CIVIL,QUANT_DEPENDENTES,NIVEL_EDUCACIONAL,NACIONALIDADE,TIPO_RESIDENCIA,MESES_RESIDENCIA,FLAG_EMAIL,RENDA_PESSOAL_MENSAL,OUTRAS_RENDAS,FLAG_VISA,FLAG_MASTERCARD,FLAG_DINERS,FLAG_AMERICAN_EXPRESS,FLAG_OUTROS_CARTOES,QUANT_CONTAS_BANCARIAS,QUANT_CONTAS_BANCARIAS_ESPECIAIS,VALOR_PATRIMONIO_PESSOAL,QUANT_CARROS,MESES_NO_TRABALHO,CODIGO_PROFISSAO,TIPO_OCUPACAO,CODIGO_PROFISSAO_CONJUGE,NIVEL_EDUCACIONAL_CONJUGE,FLAG_DOCUMENTO_RESIDENCIAL,FLAG_RG,FLAG_CPF,FLAG_COMPROVANTE_RENDA,PRODUTO,IDADE,ROTULO_ALVO_MAU=1
count,49999.000000,49999.000000,49999.0,49999.000000,49999.000000,49999.000000,49999.0,49999.000000,48650.000000,46222.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,49999.000000,4.999900e+04,49999.000000,49999.000000,42243.000000,42686.000000,21116.000000,17662.000000,49999.0,49999.0,49999.0,49999.0,49999.000000,49999.000000,49999.000000
mean,25001.000000,12.870077,0.0,1.006540,2.148323,0.650513,0.0,0.961599,1.252230,9.727035,0.802276,886.678170,35.435469,0.111422,0.097442,0.001320,0.001740,0.002040,0.357847,0.357847,2.322419e+03,0.336147,0.009320,8.061762,2.484281,3.797926,0.296003,0.0,0.0,0.0,0.0,1.275706,43.248745,0.260805
std,14433.612391,6.608357,0.0,0.080607,1.322751,1.193666,0.0,0.202107,0.867841,10.668928,0.398287,7847.037799,891.524044,0.314657,0.296562,0.036309,0.041678,0.045121,0.479955,0.479955,4.235841e+04,0.472395,0.383457,3.220139,1.532262,5.212168,0.955688,0.0,0.0,0.0,0.0,0.988295,14.989115,0.439078
min,2.000000,1.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,6.000000,0.000000
25%,12501.500000,10.000000,0.0,1.000000,1.000000,0.000000,0.0,1.000000,1.000000,1.000000,1.000000,360.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,31.000000,0.000000
50%,25001.000000,10.000000,0.0,1.000000,2.000000,0.000000,0.0,1.000000,1.000000,6.000000,1.000000,500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,9.000000,2.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,41.000000,0.000000
75%,37500.500000,15.000000,0.0,1.000000,2.000000,1.000000,0.0,1.000000,1.000000,15.000000,1.000000,800.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000e+00,1.000000,0.000000,9.000000,4.000000,11.000000,0.000000,0.0,0.0,0.0,0.0,1.000000,53.000000,1.000000
max,50000.000000,25.000000,0.0,2.000000,7.000000,53.000000,0.0,2.000000,5.000000,228.000000,1.000000,959000.000000,194344.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,6.000000e+06,1.000000,35.000000,18.000000,5.000000,17.000000,5.000000,0.0,0.0,0.0,0.0,7.000000,106.000000,1.000000


A partir da descrição estatística das variáveis, é possível notar que as variáveis quantitativas "QUANT_CARTOES_ADICIONAIS", "NIVEL_EDUCACIONAL", "FLAG_DOCUMENTO_RESIDENCIAL", "FLAG_RG", "FLAG_CPF", "FLAG_COMPROVANTE_RENDA" estão com todos seus valores nulos, e, portanto, serão descartadas.

In [17]:
#Retirar as colunas "QUANT_CARTOES_ADICIONAIS", "NIVEL_EDUCACIONAL", "FLAG_DOCUMENTO_RESIDENCIAL", "FLAG_RG"
#"FLAG_CPF", "FLAG_COMPROVANTE_RENDA" do dataset

columns_to_drop = ["QUANT_CARTOES_ADICIONAIS", "NIVEL_EDUCACIONAL", "FLAG_DOCUMENTO_RESIDENCIAL", "FLAG_RG", "FLAG_CPF", "FLAG_COMPROVANTE_RENDA"]
dataset = dataset.drop(columns=columns_to_drop)


In [18]:
# Analisar quantos valores distintos existem para as colunas categoricas

# Selecionar apenas as colunas categóricas
categorical_columns = dataset.select_dtypes(include=['object']).columns

# Iterar sobre as colunas categóricas e imprimir o número de valores distintos
for column in categorical_columns:
  print(f"Coluna: {column}, Número de valores distintos: {dataset[column].nunique()}")


Coluna: TIPO_FUNCIONARIO, Número de valores distintos: 1
Coluna: TIPO_ENVIO_APLICACAO, Número de valores distintos: 3
Coluna: SEXO, Número de valores distintos: 4
Coluna: ESTADO_NASCIMENTO, Número de valores distintos: 29
Coluna: CIDADE_NASCIMENTO, Número de valores distintos: 9910
Coluna: ESTADO_RESIDENCIAL, Número de valores distintos: 27
Coluna: CIDADE_RESIDENCIAL, Número de valores distintos: 3528
Coluna: BAIRRO_RESIDENCIAL, Número de valores distintos: 14511
Coluna: FLAG_TELEFONE_RESIDENCIAL, Número de valores distintos: 2
Coluna: CODIGO_AREA_TELEFONE_RESIDENCIAL, Número de valores distintos: 102
Coluna: FLAG_TELEFONE_MOVEL, Número de valores distintos: 1
Coluna: EMPRESA, Número de valores distintos: 2
Coluna: ESTADO_PROFISSIONAL, Número de valores distintos: 28
Coluna: CIDADE_PROFISSIONAL, Número de valores distintos: 2236
Coluna: BAIRRO_PROFISSIONAL, Número de valores distintos: 5057
Coluna: FLAG_TELEFONE_PROFISSIONAL, Número de valores distintos: 2
Coluna: CODIGO_AREA_TELEFONE_

A partir da contagem de valores distintos das variáveis categoricas, é possível notar que as colunas "TIPO_FUNCIONARIO", "FLAG_TELEFONE_MOVEL", "FLAG_REGISTRO_ACSP" estão com todos seus valores iguais, e, portanto, serão descartadas.

In [19]:
columns_cat_to_drop = ["TIPO_FUNCIONARIO", "FLAG_TELEFONE_MOVEL", "FLAG_REGISTRO_ACSP"]
dataset = dataset.drop(columns=columns_cat_to_drop)

In [24]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 0 to 49998
Data columns (total 45 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ID_CLIENTE                         49999 non-null  int64  
 1   DIA_PAGAMENTO                      49999 non-null  int64  
 2   TIPO_ENVIO_APLICACAO               49999 non-null  object 
 3   TIPO_ENDERECO_POSTAL               49999 non-null  int64  
 4   SEXO                               49999 non-null  object 
 5   ESTADO_CIVIL                       49999 non-null  int64  
 6   QUANT_DEPENDENTES                  49999 non-null  int64  
 7   ESTADO_NASCIMENTO                  49999 non-null  object 
 8   CIDADE_NASCIMENTO                  49999 non-null  object 
 9   NACIONALIDADE                      49999 non-null  int64  
 10  ESTADO_RESIDENCIAL                 49999 non-null  object 
 11  CIDADE_RESIDENCIAL                 49999 non-null  obj

**Após a limpeza inicial de dados saimos de 53 variáveis para 44, sendo uma delas o ID_Cliente que não será utilizado no modelo. Os próximos passos serão tratar, de forma simples e rápida, os dados nulos e os dados existentes para aplicar um modelo baseline**

## **Análise de Dados Nulos**

In [30]:
# Calcula a quantidade de valores nulos para cada coluna
null_counts = dataset.isnull().sum()

# Filtra apenas as colunas que possuem pelo menos um valor nulo
columns_with_nulls = null_counts[null_counts > 0]

# Cria um novo DataFrame com as colunas que possuem valores nulos
df_nulls = pd.DataFrame({'Quantidade de Nulos': columns_with_nulls})

# Calcula a porcentagem de valores nulos para cada coluna
df_nulls['Porcentagem de Nulos (%)'] = (df_nulls['Quantidade de Nulos'] / len(dataset)) * 100

# Imprime o DataFrame com a quantidade e a porcentagem de valores nulos para cada coluna
print("Colunas com valores nulos:")
print(df_nulls)


Colunas com valores nulos:
                           Quantidade de Nulos  Porcentagem de Nulos (%)
TIPO_RESIDENCIA                           1349                  2.698054
MESES_RESIDENCIA                          3777                  7.554151
CIDADE_PROFISSIONAL                      33782                 67.565351
BAIRRO_PROFISSIONAL                      33782                 67.565351
CODIGO_PROFISSAO                          7756                 15.512310
TIPO_OCUPACAO                             7313                 14.626293
CODIGO_PROFISSAO_CONJUGE                 28883                 57.767155
NIVEL_EDUCACIONAL_CONJUGE                32337                 64.675294


Nota-se que 8 variáveis apresentam valores nulos

In [ ]:
# Aqui pra baixo, até a análise dos dados existentes devemos definir uma forma simples (Baseline) para tratar os missing values.
# Foi uma exigência pro professor que apresentemos um modelo baseline com definições simples de feature engineering a fim de ter
# um primeiro resultado como ponto de partida

## **Análise dos Dados Existentes**

_Para as variáveis onde há todos os dados, é necessário entender se os dados existentes fazem sentido_

In [46]:
# Criar um novo dataset apenas com as colunas que não possuem nenhum valor nulo
dataset_sem_nulos = dataset.dropna(axis=1, how='any')

# Imprimir informações sobre o novo dataset
print("Dataset sem colunas com valores nulos:")
dataset_sem_nulos.info()

Dataset sem colunas com valores nulos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49999 entries, 0 to 49998
Data columns (total 37 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ID_CLIENTE                         49999 non-null  int64  
 1   DIA_PAGAMENTO                      49999 non-null  int64  
 2   TIPO_ENVIO_APLICACAO               49999 non-null  object 
 3   TIPO_ENDERECO_POSTAL               49999 non-null  int64  
 4   SEXO                               49999 non-null  object 
 5   ESTADO_CIVIL                       49999 non-null  int64  
 6   QUANT_DEPENDENTES                  49999 non-null  int64  
 7   ESTADO_NASCIMENTO                  49999 non-null  object 
 8   CIDADE_NASCIMENTO                  49999 non-null  object 
 9   NACIONALIDADE                      49999 non-null  int64  
 10  ESTADO_RESIDENCIAL                 49999 non-null  object 
 11  CIDADE_RESIDENC

Vamos começar analisando as variáveis binárias e categoricas. Variáveis como renda e patrimônio possuem muitos valores possíveis e sua coerência será analisada posteriormente a partir de análises de outliers. Enquanto as variáveis de texto como cidade e bairro não serão analisadas por existir inumeras possibilidades. As variáveis e CEP serão analisadas isoladamente

### **Análise Binárias e Categoricas**

In [47]:
columns_to_drop_x = ["ID_CLIENTE", "CIDADE_NASCIMENTO", "CIDADE_RESIDENCIAL", "BAIRRO_RESIDENCIAL", "RENDA_PESSOAL_MENSAL", "OUTRAS_RENDAS", "VALOR_PATRIMONIO_PESSOAL", "CEP_RESIDENCIAL_3", "CEP_PROFISSIONAL_3"]
columns_without_nulls = dataset_sem_nulos.drop(columns=columns_to_drop_x)

In [48]:
# Iterar sobre as colunas e imprimir os valores distintos
for column in columns_without_nulls.columns:
    print(f"Coluna: {column}")
    print(columns_without_nulls[column].unique())
    print("-" * 20)

Coluna: DIA_PAGAMENTO
[15  5 20 10 25  1]
--------------------
Coluna: TIPO_ENVIO_APLICACAO
['Carga' 'Web' '0']
--------------------
Coluna: TIPO_ENDERECO_POSTAL
[1 2]
--------------------
Coluna: SEXO
['F' 'M' 'N' ' ']
--------------------
Coluna: ESTADO_CIVIL
[2 1 5 3 7 4 6 0]
--------------------
Coluna: QUANT_DEPENDENTES
[ 0  2  1  3  4  5  6  7 10  8 13 11  9 12 14 15 53]
--------------------
Coluna: ESTADO_NASCIMENTO
['RJ' 'RN' 'PE' 'MG' 'BA' 'SP' 'RS' 'CE' 'PA' 'PB' 'MA' ' ' 'GO' 'AC' 'MT'
 'AL' 'AP' 'TO' 'SC' 'PR' 'MS' 'DF' 'PI' 'RO' 'ES' 'AM' 'SE' 'RR' 'XX']
--------------------
Coluna: NACIONALIDADE
[1 0 2]
--------------------
Coluna: ESTADO_RESIDENCIAL
['RJ' 'RN' 'PE' 'MG' 'BA' 'SP' 'RS' 'CE' 'AP' 'MS' 'DF' 'PB' 'MA' 'PA'
 'GO' 'PR' 'MT' 'AL' 'ES' 'SC' 'AM' 'RO' 'PI' 'RR' 'TO' 'SE' 'AC']
--------------------
Coluna: FLAG_TELEFONE_RESIDENCIAL
['Y' 'N']
--------------------
Coluna: CODIGO_AREA_TELEFONE_RESIDENCIAL
['20' '105' ' ' '33' '81' '12' '54' '107' '32' '39' '5' '24' '

### **Análise Rendas**

### **Análise CEPs**

## **Modelo Baseline**